In [4]:
pip install  xgboost pandas scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [1]:
import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

In [4]:
import flwr as fl
import pickle
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [3]:
# Load dataset
client_data = pd.read_csv("young_adults.csv")  # Change this for each client
X = client_data.drop(columns=["TenYearCHD"]).values  # Features
y = client_data["TenYearCHD"].values  # Target

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to PyTorch tensors
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.long), torch.tensor(y_test, dtype=torch.long)

# Create DataLoaders
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32)

# Define MLP Model
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  # Output layer for binary classification
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

# Initialize model, loss, and optimizer
model = MLP(input_size=X.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Flower client
class MLPClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        return [param.data.numpy() for param in model.parameters()]

    def set_parameters(self, parameters):
        for param, new_param in zip(model.parameters(), parameters):
            param.data = torch.tensor(new_param, dtype=torch.float32)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        model.train()
        for epoch in range(5):  # Local training for 5 epochs
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                y_pred = model(X_batch)
                loss = criterion(y_pred, y_batch)
                loss.backward()
                optimizer.step()
        return self.get_parameters(config), len(X_train), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                y_pred = model(X_batch)
                _, predicted = torch.max(y_pred, 1)
                correct += (predicted == y_batch).sum().item()
                total += y_batch.size(0)
        acc = correct / total
        return 1 - acc, len(X_test), {}

# Start Flower client
fl.client.start_numpy_client(server_address="127.0.0.1:8081", client=MLPClient())

KeyboardInterrupt: 

In [11]:
import flwr as fl
import xgboost as xgb
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load local dataset
data = pd.read_csv("young_adults.csv")  # Change file for each client

# Prepare data
X = data.drop(columns=["TenYearCHD"])
y = data["TenYearCHD"]

# Split into training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize XGBoost model
model = xgb.XGBClassifier(n_estimators=50, max_depth=3)
model.fit(X_train, y_train)

# Check initial accuracy before training
y_pred = model.predict(X_test)
local_acc = accuracy_score(y_test, y_pred)
print(f"⚡ Client Initial Accuracy: {local_acc:.4f}")

# Define Flower Client
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model):
        self.model = model

    def get_parameters(self):
        """Extract model parameters and convert them to bytes."""
        model_bytes = pickle.dumps(self.model)  # Serialize model
        print(f"📤 Sending Parameters to Server: {len(model_bytes)} bytes")  # Debugging
        return [model_bytes]

    def set_parameters(self, parameters):
        """Set model parameters received from the server."""
        self.model = pickle.loads(parameters[0])  # Deserialize model
        print("✅ Model parameters updated from the server.")

    def fit(self, parameters, config):
        """Train model on local client data."""
        self.set_parameters(parameters)
        print("📥 Received parameters from server.")  

        # Train local model
        self.model.fit(X_train, y_train)

        return self.get_parameters(), len(X_train), {}

    def evaluate(self, parameters, config):
        """Evaluate model on the local test set."""
        self.set_parameters(parameters)
        print("📊 Evaluating local model.")

        y_pred = self.model.predict(X_test)
        local_acc = accuracy_score(y_test, y_pred)

        return float(local_acc), len(X_test), {}

# Start Flower client
fl.client.start_numpy_client(server_address="127.0.0.1:8081", client=FlowerClient(model))


INFO flwr 2025-03-13 22:45:40,733 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2025-03-13 22:45:40,761 | connection.py:39 | ChannelConnectivity.CONNECTING
DEBUG flwr 2025-03-13 22:45:40,766 | connection.py:39 | ChannelConnectivity.READY


⚡ Client Initial Accuracy: 0.9554


DEBUG flwr 2025-03-13 22:47:44,788 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flwr 2025-03-13 22:47:44,997 | connection.py:113 | gRPC channel closed


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "IOCP/Socket: Connection reset (An existing connection was forcibly closed by the remote host.
 -- 10054)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"IOCP/Socket: Connection reset (An existing connection was forcibly closed by the remote host.\r\n -- 10054)", grpc_status:14, created_time:"2025-03-13T17:17:44.7639377+00:00"}"
>

In [3]:
import flwr as fl
import numpy as np
import xgboost as xgb

# Create a Dummy XGBoost Model
dummy_model = xgb.XGBClassifier(n_estimators=10, use_label_encoder=False, eval_metric="logloss")

# Generate Random Model Parameters
def generate_dummy_parameters():
    return [np.random.rand(10, 10).astype(np.float32) for _ in range(5)]

# Define Dummy Flower Client
class DummyClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        """Return fake model parameters."""
        print("📤 Sending dummy parameters to server...")
        return generate_dummy_parameters()

    def set_parameters(self, parameters):
        """Receive and ignore model parameters from server."""
        print("📥 Received parameters from server (not used in dummy client).")

    def fit(self, parameters, config):
        """Simulate local training by just returning dummy parameters."""
        self.set_parameters(parameters)
        print("🔄 Dummy client training (simulated)...")
        return self.get_parameters(config), 100, {}

    def evaluate(self, parameters, config):
        """Simulate evaluation by returning a random accuracy."""
        self.set_parameters(parameters)
        accuracy = np.random.uniform(0.7, 1.0)  # Random accuracy for testing
        print(f"📊 Dummy evaluation result: {accuracy:.2f}")
        return float(accuracy), 100, {}

# Start the Dummy Client
fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=DummyClient())


	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      
INFO :      Received: train message eb561eb4-eee6-4b19-afc2-55f5f58a6793
INFO :      Sent reply


📥 Received parameters from server (not used in dummy client).
🔄 Dummy client training (simulated)...
📤 Sending dummy parameters to server...


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "IOCP/Socket: Connection reset (An existing connection was forcibly closed by the remote host.
 -- 10054)"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:127.0.0.1:8080 {grpc_message:"IOCP/Socket: Connection reset (An existing connection was forcibly closed by the remote host.\r\n -- 10054)", grpc_status:14, created_time:"2025-03-14T06:58:57.3163556+00:00"}"
>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import flwr as fl
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from torch.utils.data import DataLoader, TensorDataset

# Load dataset
df = pd.read_csv("middle_aged_adults.csv")
X = df.iloc[:, :-1].values  # Features
y = df.iloc[:, -1].values   # Target

# Balance dataset using SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64)

# Define MLP Model
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return torch.sigmoid(self.fc3(x))

# Initialize model
input_size = X_train.shape[1]  
model = MLP(input_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00545)

# Define Flower client, explicitly setting this as Client 2
class FLClient(fl.client.NumPyClient):
    def __init__(self):
        self.cid = 2  # Set this client as Client 2

    def get_parameters(self, config):
        print(f"[Client {self.cid}] Sending model parameters")
        return [val.cpu().detach().numpy() for val in model.state_dict().values()]
    
    def set_parameters(self, parameters):
        print(f"[Client {self.cid}] Receiving model parameters")
        param_dict = zip(model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(v) for k, v in param_dict}
        model.load_state_dict(state_dict, strict=True)
    
    def fit(self, parameters, config):
        self.set_parameters(parameters)
        model.train()
        for epoch in range(10):
            total_loss = 0
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            print(f"[Client {self.cid}] Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")
        return self.get_parameters(config), len(X_train), {}
    
    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        model.eval()
        correct, total, loss = 0, 0, 0.0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                outputs = model(X_batch)
                loss += criterion(outputs, y_batch).item()
                predictions = (outputs > 0.5).float()
                correct += (predictions == y_batch).sum().item()
                total += y_batch.size(0)
        
        accuracy = correct / total if total > 0 else 0.0
        print(f"[Client {self.cid}] Accuracy: {accuracy:.4f}")
        return float(loss), len(X_test), {"accuracy": accuracy}

# Start the client (Explicitly as Client 2)
fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=FLClient())


	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.
 -- 10061)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.\r\n -- 10061)", grpc_status:14, created_time:"2025-03-16T11:52:04.3641579+00:00"}"
>

In [20]:
import pandas as pd

df_young = pd.read_csv("young_adults.csv")
df_middle = pd.read_csv("middle_aged_adults.csv")
df_older = pd.read_csv("older_adults.csv")

print("🔹 Young Adults:", df_young.shape)
print("🔹 Middle-Aged Adults:", df_middle.shape)
print("🔹 Older Adults:", df_older.shape)


🔹 Young Adults: (556, 16)
🔹 Middle-Aged Adults: (2994, 16)
🔹 Older Adults: (690, 16)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import flwr as fl
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from torch.utils.data import DataLoader, TensorDataset

# Load dataset
df = pd.read_csv("older_adults.csv")
X = df.iloc[:, :-1].values  # Features
y = df.iloc[:, -1].values   # Target

# Balance dataset using SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64)

# Define Fine-Tuned MLP Model
class MLP(nn.Module):
    def __init__(self, input_size, hidden1=512, hidden2=512, hidden3=256, hidden4=128, output_size=1, dropout_prob=0.2):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1)
        self.bn1 = nn.BatchNorm1d(hidden1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)

        self.fc2 = nn.Linear(hidden1, hidden2)
        self.bn2 = nn.BatchNorm1d(hidden2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)

        self.fc3 = nn.Linear(hidden2, hidden3)
        self.bn3 = nn.BatchNorm1d(hidden3)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_prob)

        self.fc4 = nn.Linear(hidden3, hidden4)
        self.bn4 = nn.BatchNorm1d(hidden4)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(dropout_prob)

        self.fc5 = nn.Linear(hidden4, output_size)

    def forward(self, x):
        x = self.relu1(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = self.relu2(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = self.relu3(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        x = self.relu4(self.bn4(self.fc4(x)))
        x = self.dropout4(x)
        x = torch.sigmoid(self.fc5(x))
        return x

# Initialize Model
input_size = X_train.shape[1]  
model = MLP(input_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0008, weight_decay=5e-4)

# Load Fine-Tuned Model If Available
try:
    model.load_state_dict(torch.load("best_model.pth"))
    print("✅ Loaded fine-tuned model weights!")
except FileNotFoundError:
    print("⚠️ Fine-tuned model not found. Using default initialization.")

# Define Flower Client
class FLClient(fl.client.NumPyClient):
    def __init__(self):
        self.cid = 1  # Always set this client as Client 1

    def get_parameters(self, config):
        print(f"[Client {self.cid}] Sending model parameters")
        return [val.cpu().detach().numpy() for val in model.state_dict().values()]
    
    def set_parameters(self, parameters):
        print(f"[Client {self.cid}] Receiving model parameters")
        param_dict = zip(model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(v) for k, v in param_dict}
        model.load_state_dict(state_dict, strict=True)
    
    def fit(self, parameters, config):
        self.set_parameters(parameters)
        model.train()
        for epoch in range(10):
            total_loss = 0
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            print(f"[Client {self.cid}] Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")
        return self.get_parameters(config), len(X_train), {}
    
    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        model.eval()
        correct, total, loss = 0, 0, 0.0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                outputs = model(X_batch)
                loss += criterion(outputs, y_batch).item()
                predictions = (outputs > 0.5).float()
                correct += (predictions == y_batch).sum().item()
                total += y_batch.size(0)
        
        accuracy = correct / total if total > 0 else 0.0
        print(f"[Client {self.cid}] Accuracy: {accuracy:.4f}")
        return float(loss), len(X_test), {"accuracy": accuracy}

# Start the client (Always as Client 1)
fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=FLClient())


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import ADASYN
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Load dataset
df = pd.read_csv("older_adults.csv")  # Ensure correct dataset path

# Feature selection using RandomForest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
X = df.drop(columns=["TenYearCHD"])  # Drop target variable
y = df["TenYearCHD"]  # Target variable

# Train feature selection model
rf.fit(X, y)
feature_importances = rf.feature_importances_

# Select top 10 most important features
N = 10
important_features = X.columns[feature_importances.argsort()[-N:]]
X_selected = df[important_features]

# Apply ADASYN for class balancing
X_resampled, y_resampled = ADASYN().fit_resample(X_selected, y)

# Split dataset (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Convert to tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.long)

# Create DataLoaders
batch_size = 128
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size, shuffle=False)

# Define Neural Network for TenYearCHD Prediction
class CHDPredictionMLP(nn.Module):
    def __init__(self, input_size):
        super(CHDPredictionMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(512, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)

        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.2)

        self.fc4 = nn.Linear(256, 128)
        self.bn4 = nn.BatchNorm1d(128)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(0.2)

        self.fc5 = nn.Linear(128, 2)  # Binary classification
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.dropout3(x)

        x = self.fc4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.dropout4(x)

        x = self.fc5(x)
        return self.softmax(x)

# Initialize Model
input_size = X_selected.shape[1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CHDPredictionMLP(input_size).to(device)

# Loss, Optimizer, and Learning Rate Scheduling
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0008, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.001, step_size_up=len(train_loader) * 10, mode='triangular')
scheduler2 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=20)

# Training with Accuracy Evaluation
epochs = 50

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_loss += loss.item()

    # Validation Accuracy
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

    val_acc = correct / total if total > 0 else 0.0
    scheduler2.step(val_acc)

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {train_loss/len(train_loader):.4f}, Validation Accuracy: {val_acc:.4f}")

# Save the trained model
torch.save(model.state_dict(), "trained_chd_model.pth")
print("✅ Model training complete. Saved as 'trained_chd_model.pth'")


Epoch [1/50], Loss: 0.6885, Validation Accuracy: 0.5590
Epoch [2/50], Loss: 0.6804, Validation Accuracy: 0.5282
Epoch [3/50], Loss: 0.6674, Validation Accuracy: 0.5179
Epoch [4/50], Loss: 0.6703, Validation Accuracy: 0.5077
Epoch [5/50], Loss: 0.6520, Validation Accuracy: 0.4974
Epoch [6/50], Loss: 0.6443, Validation Accuracy: 0.5026
Epoch [7/50], Loss: 0.6307, Validation Accuracy: 0.5590
Epoch [8/50], Loss: 0.6271, Validation Accuracy: 0.5538
Epoch [9/50], Loss: 0.6260, Validation Accuracy: 0.5590
Epoch [10/50], Loss: 0.6607, Validation Accuracy: 0.5641
Epoch [11/50], Loss: 0.6416, Validation Accuracy: 0.5897
Epoch [12/50], Loss: 0.6272, Validation Accuracy: 0.5744
Epoch [13/50], Loss: 0.6430, Validation Accuracy: 0.5795
Epoch [14/50], Loss: 0.6223, Validation Accuracy: 0.6103
Epoch [15/50], Loss: 0.6350, Validation Accuracy: 0.6256
Epoch [16/50], Loss: 0.6036, Validation Accuracy: 0.6103
Epoch [17/50], Loss: 0.6274, Validation Accuracy: 0.5846
Epoch [18/50], Loss: 0.5973, Validation 

today

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import flwr as fl
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from torch.utils.data import DataLoader, TensorDataset

# Load dataset
df = pd.read_csv("middle_aged_adults.csv")
X = df.iloc[:, :-1].values  # Features
y = df.iloc[:, -1].values   # Target

# Balance dataset using SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Changed to long for classification
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoaders
batch_size = 128
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size, shuffle=False)

# Define the Fine-Tuned MLP Model
class MLP(nn.Module):
    def __init__(self, input_size, hidden1=512, hidden2=512, hidden3=256, hidden4=128, output_size=2, dropout_prob=0.2):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1)
        self.bn1 = nn.BatchNorm1d(hidden1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)

        self.fc2 = nn.Linear(hidden1, hidden2)
        self.bn2 = nn.BatchNorm1d(hidden2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)

        self.fc3 = nn.Linear(hidden2, hidden3)
        self.bn3 = nn.BatchNorm1d(hidden3)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_prob)

        self.fc4 = nn.Linear(hidden3, hidden4)
        self.bn4 = nn.BatchNorm1d(hidden4)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(dropout_prob)

        self.fc5 = nn.Linear(hidden4, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.dropout3(x)

        x = self.fc4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.dropout4(x)

        x = self.fc5(x)
        return x

# Initialize model
input_size = X_train.shape[1]  
model = MLP(input_size).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0008, weight_decay=5e-4)  # L2 regularization

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.CyclicLR(
    optimizer, 
    base_lr=0.0001,  
    max_lr=0.001,  
    step_size_up=len(train_loader) * 10,  
    mode='triangular'
)

scheduler2 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=20)

# Define Flower Client
class FLClient(fl.client.NumPyClient):
    def __init__(self):
        self.cid = 1  

    def wait_for_server_ready(self, host="127.0.0.1", port=8081):
        """Wait for the aggregation server to signal readiness before starting training."""
        print("🔄 Waiting for the aggregation server to be ready...")
        
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            try:
                sock.connect((host, port))
                sock.sendall(b"PONG")  # Respond to PING
                msg = sock.recv(1024)  # Wait for READY signal

                if msg.strip() == b"READY":
                    print("✅ Aggregation server is ready! Sending ACK...")
                    sock.sendall(b"ACK")  # Confirm readiness
                else:
                    print("❌ Did not receive READY signal. Exiting.")
                    exit(1)

            except ConnectionRefusedError:
                print("❌ Could not connect to the aggregation server. Exiting.")
                exit(1)

    def fit(self, parameters, config):
        """Train the model using received parameters."""
        self.set_parameters(parameters)

        print(f"[Client {self.cid}] Starting training...")
        model.train()

        for epoch in range(10):
            total_loss = 0
            for X_batch, y_batch in train_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)

                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
            
            print(f"[Client {self.cid}] Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

        return self.get_parameters(config), len(X_train), {}

# Before starting training, wait for the server to be ready
client = FLClient()
client.wait_for_server_ready()

# Start the client
fl.client.start_numpy_client(server_address="127.0.0.1:8081", client=client)



	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8081: connection attempt timed out before receiving SETTINGS frame"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-03-16T13:54:43.8133253+00:00", grpc_status:14, grpc_message:"failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8081: connection attempt timed out before receiving SETTINGS frame"}"
>

: 

cient 2 form helthsheild


In [ ]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR
import opacus
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import time
import os
import json
from typing import Dict, List, Tuple

In [13]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import os
import logging

In [19]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import os
import sys
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("FraminghamClient")

# Client configuration
CLIENT_CONFIG = {
    "local_epochs": 3,
    "batch_size": 32,
    "learning_rate": 0.001,
    "weight_decay": 1e-5,
    "dropout_rate": 0.3,
    "server_address": "localhost:8080"
}

# Model for Framingham Heart Study data
class HeartDiseaseModel(nn.Module):
    def __init__(self, input_size):
        super(HeartDiseaseModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(CLIENT_CONFIG["dropout_rate"]),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(CLIENT_CONFIG["dropout_rate"]),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

# Load and preprocess Framingham data
def load_data(data_path):
    """Load and preprocess Framingham Heart Study data"""
    try:
        # Read CSV data
        df = pd.read_csv(data_path)
        logger.info(f"Loaded {data_path} with shape {df.shape}")
        
        # Check for missing values
        missing_values = df.isnull().sum().sum()
        if missing_values > 0:
            logger.info(f"Found {missing_values} missing values, dropping rows with missing values")
            df.dropna(inplace=True)
            logger.info(f"Shape after dropping missing values: {df.shape}")
        
        # Ensure the target column exists
        if "TenYearCHD" not in df.columns:
            raise ValueError("Target column 'TenYearCHD' not found in dataset!")
            
        # Split features and target
        X = df.drop(columns=["TenYearCHD"])
        y = df["TenYearCHD"]
        
        # Show class distribution
        logger.info(f"Class distribution: {y.value_counts().to_dict()}")
        
        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Convert to tensors
        X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
        y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)
        
        # Create dataset and dataloader
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(dataset, batch_size=CLIENT_CONFIG["batch_size"], shuffle=True)
        
        logger.info(f"Created dataloader with {len(dataset)} samples and {X.shape[1]} features")
        return dataloader, X.shape[1]
    
    except Exception as e:
        logger.error(f"Error loading data: {str(e)}")
        raise

# Client class for Federated Learning
class FraminghamClient(fl.client.NumPyClient):
    def __init__(self, model, dataloader, device):
        self.model = model
        self.dataloader = dataloader
        self.device = device
        logger.info(f"Initialized client with device: {device}")
        
    def get_parameters(self, config):
        """Get model parameters as a list of NumPy arrays"""
        # Using detach() to prevent gradient error
        return [val.detach().cpu().numpy() for val in self.model.parameters()]
    
    def set_parameters(self, parameters):
        """Set model parameters from a list of NumPy arrays"""
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(v) for k, v in params_dict}
        self.model.load_state_dict(state_dict, strict=True)
        logger.info("Parameters updated from server")
        
    def fit(self, parameters, config):
        """Train the model on local data"""
        # Update model with server parameters
        self.set_parameters(parameters)
        
        # Train the model
        self.model.train()
        optimizer = torch.optim.Adam(
            self.model.parameters(), 
            lr=CLIENT_CONFIG["learning_rate"],
            weight_decay=CLIENT_CONFIG["weight_decay"]
        )
        criterion = nn.BCELoss()
        
        # Metrics for tracking
        total_loss = 0.0
        total_samples = 0
        correct = 0
        
        # Train for multiple epochs
        for epoch in range(CLIENT_CONFIG["local_epochs"]):
            epoch_loss = 0.0
            epoch_samples = 0
            
            for batch_idx, (X, y) in enumerate(self.dataloader):
                # Move tensors to device
                X, y = X.to(self.device), y.to(self.device)
                
                # Forward pass
                y_pred = self.model(X)
                loss = criterion(y_pred, y)
                
                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                # Update metrics
                batch_loss = loss.item() * X.size(0)
                total_loss += batch_loss
                epoch_loss += batch_loss
                total_samples += X.size(0)
                epoch_samples += X.size(0)
                
                # Calculate accuracy
                predicted = (y_pred > 0.5).float()
                correct += (predicted == y).sum().item()
                
                # Log progress occasionally
                if batch_idx % 5 == 0:
                    logger.info(
                        f"Epoch {epoch+1}/{CLIENT_CONFIG['local_epochs']} - "
                        f"Batch {batch_idx}/{len(self.dataloader)} - "
                        f"Loss: {loss.item():.4f}"
                    )
            
            # Log epoch metrics
            logger.info(
                f"Epoch {epoch+1}/{CLIENT_CONFIG['local_epochs']} completed - "
                f"Loss: {epoch_loss/epoch_samples:.4f}"
            )
        
        # Calculate final metrics
        avg_loss = total_loss / total_samples if total_samples > 0 else 0
        accuracy = correct / total_samples if total_samples > 0 else 0
        
        logger.info(f"Training completed - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
        
        # Return updated model parameters and metrics
        return self.get_parameters({}), total_samples, {"loss": float(avg_loss), "accuracy": float(accuracy)}
    
    def evaluate(self, parameters, config):
        """Evaluate the model on local data"""
        # Update model with server parameters
        self.set_parameters(parameters)
        
        # Evaluate the model
        self.model.eval()
        criterion = nn.BCELoss()
        
        loss = 0.0
        total = 0
        correct = 0
        
        with torch.no_grad():
            for X, y in self.dataloader:
                # Move tensors to device
                X, y = X.to(self.device), y.to(self.device)
                
                # Forward pass
                y_pred = self.model(X)
                batch_loss = criterion(y_pred, y).item()
                
                # Update metrics
                loss += batch_loss * X.size(0)
                total += X.size(0)
                
                # Calculate accuracy
                predicted = (y_pred > 0.5).float()
                correct += (predicted == y).sum().item()
        
        # Calculate final metrics
        avg_loss = loss / total if total > 0 else 0
        accuracy = correct / total if total > 0 else 0
        
        logger.info(f"Evaluation - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
        
        # Return metrics
        return float(avg_loss), total, {"accuracy": float(accuracy)}

def start_client(client_id=0, server_address=None):
    """Initialize and start a client"""
    # Update server address if provided
    if server_address:
        CLIENT_CONFIG["server_address"] = server_address
    
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")
    
    # Determine which data file to use based on client ID
    data_path = f"framingham_part{client_id+1}.csv"
    
    # Try alternative naming if file doesn't exist
    if not os.path.exists(data_path):
        alternative_path = f"framingham_part{client_id+1}.csv"
        if os.path.exists(alternative_path):
            data_path = alternative_path
        else:
            logger.error(f"Data file {data_path} not found")
            return
    
    # Load data
    dataloader, input_size = load_data(data_path)
    
    # Initialize model
    model = HeartDiseaseModel(input_size=input_size).to(device)
    logger.info(f"Model initialized with input size: {input_size}")
    
    # Create client
    client = FraminghamClient(model, dataloader, device)
    
    # Start client
    logger.info(f"Starting client {client_id} and connecting to {CLIENT_CONFIG['server_address']}")
    
    print(f"\n===== Framingham Heart Study FL Client {client_id} =====")
    print(f"Server:        {CLIENT_CONFIG['server_address']}")
    print(f"Data file:     {data_path}")
    print(f"Local epochs:  {CLIENT_CONFIG['local_epochs']}")
    print(f"Batch size:    {CLIENT_CONFIG['batch_size']}")
    print(f"Device:        {device}")
    print("==============================================")
    print(f"\nConnecting to server...\n")
    
    fl.client.start_client(server_address=CLIENT_CONFIG["server_address"], client=client)

# For Jupyter usage
if __name__ == "__main__":
    # Check if running in Jupyter
    if 'ipykernel' in sys.modules:
        print("Running in Jupyter/IPython environment")
        # Default to client ID 0, can be changed by user
        start_client(client_id=1)
    else:
        # For command line use
        import argparse
        parser = argparse.ArgumentParser(description="Framingham Heart Study FL Client")
        parser.add_argument("--id", type=int, default=0, help="Client ID (0, 1, or 2)")
        parser.add_argument("--server", type=str, default="localhost:8080", help="Server address")
        
        args = parser.parse_args()
        
        if args.id not in [0, 1, 2]:
            logger.error("Client ID must be 0, 1, or 2")
        else:
            try:
                start_client(args.id, args.server)
            except Exception as e:
                logger.error(f"Client failed: {str(e)}")

2025-05-10 12:24:39,414 - FraminghamClient - INFO - Using device: cpu
2025-05-10 12:24:39,421 - FraminghamClient - INFO - Loaded framingham_part2.csv with shape (1060, 16)
2025-05-10 12:24:39,428 - FraminghamClient - INFO - Class distribution: {0: 914, 1: 146}
2025-05-10 12:24:39,434 - FraminghamClient - INFO - Created dataloader with 1060 samples and 15 features
2025-05-10 12:24:39,441 - FraminghamClient - INFO - Model initialized with input size: 15
2025-05-10 12:24:39,441 - FraminghamClient - INFO - Initialized client with device: cpu
2025-05-10 12:24:39,441 - FraminghamClient - INFO - Starting client 1 and connecting to localhost:8080
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions

Running in Jupyter/IPython environment

===== Framingham Heart Study FL Client 1 =====
Server:        localhost:8080
Data file:     framingham_part2.csv
Local epochs:  3
Batch size:    32
Device:        cpu

Connecting to server...



2025-05-10 12:24:39,609 - FraminghamClient - INFO - Epoch 1/3 - Batch 5/34 - Loss: 0.7184
2025-05-10 12:24:39,663 - FraminghamClient - INFO - Epoch 1/3 - Batch 10/34 - Loss: 0.6951
2025-05-10 12:24:39,713 - FraminghamClient - INFO - Epoch 1/3 - Batch 15/34 - Loss: 0.6728
2025-05-10 12:24:39,769 - FraminghamClient - INFO - Epoch 1/3 - Batch 20/34 - Loss: 0.6304
2025-05-10 12:24:39,819 - FraminghamClient - INFO - Epoch 1/3 - Batch 25/34 - Loss: 0.5515
2025-05-10 12:24:39,867 - FraminghamClient - INFO - Epoch 1/3 - Batch 30/34 - Loss: 0.6551
2025-05-10 12:24:39,894 - FraminghamClient - INFO - Epoch 1/3 completed - Loss: 0.6596
2025-05-10 12:24:39,915 - FraminghamClient - INFO - Epoch 2/3 - Batch 0/34 - Loss: 0.5208
2025-05-10 12:24:39,965 - FraminghamClient - INFO - Epoch 2/3 - Batch 5/34 - Loss: 0.4830
2025-05-10 12:24:40,014 - FraminghamClient - INFO - Epoch 2/3 - Batch 10/34 - Loss: 0.4913
2025-05-10 12:24:40,065 - FraminghamClient - INFO - Epoch 2/3 - Batch 15/34 - Loss: 0.4383
2025-0

In [27]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import os
import sys
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("FraminghamClient")

# Client configuration
CLIENT_CONFIG = {
    "local_epochs": 3,
    "batch_size": 32,
    "learning_rate": 0.001,
    "weight_decay": 1e-5,
    "dropout_rate": 0.3,
    "server_address": "localhost:8080",
    "proximal_mu": 0.01  # FedProx hyperparameter (controls the proximal term strength)
}

# Model for Framingham Heart Study data
class HeartDiseaseModel(nn.Module):
    def __init__(self, input_size):
        super(HeartDiseaseModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(CLIENT_CONFIG["dropout_rate"]),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(CLIENT_CONFIG["dropout_rate"]),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

# FedProx Loss Function
class FedProxLoss(nn.Module):
    def __init__(self, base_criterion, mu=0.01):
        super(FedProxLoss, self).__init__()
        self.base_criterion = base_criterion
        self.mu = mu  # Proximal term coefficient
        
    def forward(self, y_pred, y_true, model_params, global_params):
        # Calculate the base loss (e.g., BCE loss)
        base_loss = self.base_criterion(y_pred, y_true)
        
        # Calculate the proximal term if global parameters are provided
        proximal_term = 0.0
        if global_params is not None:
            # Sum up the squared L2 norm of the difference between local and global model parameters
            for local_param, global_param in zip(model_params, global_params):
                proximal_term += torch.sum((local_param - global_param) ** 2)
                
            # Add the weighted proximal term to the base loss
            loss = base_loss + (self.mu / 2) * proximal_term
            return loss
        
        # If no global parameters are provided, just return the base loss
        return base_loss

# Load and preprocess Framingham data
def load_data(data_path):
    """Load and preprocess Framingham Heart Study data"""
    try:
        # Read CSV data
        df = pd.read_csv(data_path)
        logger.info(f"Loaded {data_path} with shape {df.shape}")
        
        # Check for missing values
        missing_values = df.isnull().sum().sum()
        if missing_values > 0:
            logger.info(f"Found {missing_values} missing values, dropping rows with missing values")
            df.dropna(inplace=True)
            logger.info(f"Shape after dropping missing values: {df.shape}")
        
        # Ensure the target column exists
        if "TenYearCHD" not in df.columns:
            raise ValueError("Target column 'TenYearCHD' not found in dataset!")
            
        # Split features and target
        X = df.drop(columns=["TenYearCHD"])
        y = df["TenYearCHD"]
        
        # Show class distribution
        logger.info(f"Class distribution: {y.value_counts().to_dict()}")
        
        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Convert to tensors
        X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
        y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)
        
        # Create dataset and dataloader
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(dataset, batch_size=CLIENT_CONFIG["batch_size"], shuffle=True)
        
        logger.info(f"Created dataloader with {len(dataset)} samples and {X.shape[1]} features")
        return dataloader, X.shape[1]
    
    except Exception as e:
        logger.error(f"Error loading data: {str(e)}")
        raise

# Client class for Federated Learning with FedProx
class FraminghamClient(fl.client.NumPyClient):
    def __init__(self, model, dataloader, device):
        self.model = model
        self.dataloader = dataloader
        self.device = device
        self.global_params = None  # Store global model parameters for FedProx
        logger.info(f"Initialized client with device: {device}")
        
    def get_parameters(self, config):
        """Get model parameters as a list of NumPy arrays"""
        # Using detach() to prevent gradient error
        return [val.detach().cpu().numpy() for val in self.model.parameters()]
    
    def set_parameters(self, parameters):
        """Set model parameters from a list of NumPy arrays"""
        # Convert to torch tensors
        self.global_params = [torch.tensor(p, device=self.device) for p in parameters]
        
        # Update model
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(v, device=self.device) for k, v in params_dict}
        self.model.load_state_dict(state_dict, strict=True)
        logger.info("Parameters updated from server")
        
    def fit(self, parameters, config):
        """Train the model on local data with FedProx"""
        # Update model with server parameters
        self.set_parameters(parameters)
        
        # Train the model
        self.model.train()
        
        # Standard loss function
        criterion = nn.BCELoss()
        
        # FedProx loss function
        proximal_criterion = FedProxLoss(criterion, mu=CLIENT_CONFIG["proximal_mu"])
        
        optimizer = torch.optim.Adam(
            self.model.parameters(), 
            lr=CLIENT_CONFIG["learning_rate"],
            weight_decay=CLIENT_CONFIG["weight_decay"]
        )
        
        # Metrics for tracking
        total_loss = 0.0
        total_samples = 0
        correct = 0
        
        # Train for multiple epochs
        for epoch in range(CLIENT_CONFIG["local_epochs"]):
            epoch_loss = 0.0
            epoch_samples = 0
            
            for batch_idx, (X, y) in enumerate(self.dataloader):
                # Move tensors to device
                X, y = X.to(self.device), y.to(self.device)
                
                # Forward pass
                y_pred = self.model(X)
                
                # Calculate loss with proximal term
                loss = proximal_criterion(
                    y_pred, 
                    y, 
                    self.model.parameters(),  # Current model parameters
                    self.global_params        # Global model parameters
                )
                
                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                # Update metrics
                batch_loss = loss.item() * X.size(0)
                total_loss += batch_loss
                epoch_loss += batch_loss
                total_samples += X.size(0)
                epoch_samples += X.size(0)
                
                # Calculate accuracy
                predicted = (y_pred > 0.5).float()
                correct += (predicted == y).sum().item()
                
                # Log progress occasionally
                if batch_idx % 5 == 0:
                    logger.info(
                        f"Epoch {epoch+1}/{CLIENT_CONFIG['local_epochs']} - "
                        f"Batch {batch_idx}/{len(self.dataloader)} - "
                        f"Loss: {loss.item():.4f}"
                    )
            
            # Log epoch metrics
            logger.info(
                f"Epoch {epoch+1}/{CLIENT_CONFIG['local_epochs']} completed - "
                f"Loss: {epoch_loss/epoch_samples:.4f}"
            )
        
        # Calculate final metrics
        avg_loss = total_loss / total_samples if total_samples > 0 else 0
        accuracy = correct / total_samples if total_samples > 0 else 0
        
        logger.info(f"Training completed - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
        
        # Return updated model parameters and metrics
        return self.get_parameters({}), total_samples, {"loss": float(avg_loss), "accuracy": float(accuracy)}
    
    def evaluate(self, parameters, config):
        """Evaluate the model on local data"""
        # Update model with server parameters
        self.set_parameters(parameters)
        
        # Evaluate the model
        self.model.eval()
        criterion = nn.BCELoss()
        
        loss = 0.0
        total = 0
        correct = 0
        
        with torch.no_grad():
            for X, y in self.dataloader:
                # Move tensors to device
                X, y = X.to(self.device), y.to(self.device)
                
                # Forward pass
                y_pred = self.model(X)
                batch_loss = criterion(y_pred, y).item()
                
                # Update metrics
                loss += batch_loss * X.size(0)
                total += X.size(0)
                
                # Calculate accuracy
                predicted = (y_pred > 0.5).float()
                correct += (predicted == y).sum().item()
        
        # Calculate final metrics
        avg_loss = loss / total if total > 0 else 0
        accuracy = correct / total if total > 0 else 0
        
        logger.info(f"Evaluation - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
        
        # Return metrics
        return float(avg_loss), total, {"accuracy": float(accuracy)}

def start_client(client_id=0, server_address=None):
    """Initialize and start a client"""
    # Update server address if provided
    if server_address:
        CLIENT_CONFIG["server_address"] = server_address
    
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")
    
    # Determine which data file to use based on client ID
    data_path = f"framingham_part{client_id+1}.csv"
    
    # Try alternative naming if file doesn't exist
    if not os.path.exists(data_path):
        alternative_path = f"framingham_part{client_id+1}.csv"
        if os.path.exists(alternative_path):
            data_path = alternative_path
        else:
            logger.error(f"Data file {data_path} not found")
            return
    
    # Load data
    dataloader, input_size = load_data(data_path)
    
    # Initialize model
    model = HeartDiseaseModel(input_size=input_size).to(device)
    logger.info(f"Model initialized with input size: {input_size}")
    
    # Create client
    client = FraminghamClient(model, dataloader, device)
    
    # Start client
    logger.info(f"Starting client {client_id} and connecting to {CLIENT_CONFIG['server_address']}")
    
    print(f"\n===== Framingham Heart Study FL Client {client_id} (FedProx) =====")
    print(f"Server:        {CLIENT_CONFIG['server_address']}")
    print(f"Data file:     {data_path}")
    print(f"Local epochs:  {CLIENT_CONFIG['local_epochs']}")
    print(f"Batch size:    {CLIENT_CONFIG['batch_size']}")
    print(f"Proximal mu:   {CLIENT_CONFIG['proximal_mu']}")
    print(f"Device:        {device}")
    print("=================================================")
    print(f"\nConnecting to server...\n")
    
    fl.client.start_client(server_address=CLIENT_CONFIG["server_address"], client=client)

# For Jupyter usage
if __name__ == "__main__":
    # Check if running in Jupyter
    if 'ipykernel' in sys.modules:
        print("Running in Jupyter/IPython environment")
        # Default to client ID 0, can be changed by user
        start_client(client_id=1)
    else:
        # For command line use
        import argparse
        parser = argparse.ArgumentParser(description="Framingham Heart Study FL Client")
        parser.add_argument("--id", type=int, default=0, help="Client ID (0, 1, or 2)")
        parser.add_argument("--server", type=str, default="localhost:8080", help="Server address")
        parser.add_argument("--mu", type=float, default=0.01, help="FedProx proximal term strength")
        
        args = parser.parse_args()
        
        if args.id not in [0, 1, 2]:
            logger.error("Client ID must be 0, 1, or 2")
        else:
            try:
                # Set FedProx hyperparameter
                CLIENT_CONFIG["proximal_mu"] = args.mu
                
                start_client(args.id, args.server)
            except Exception as e:
                logger.error(f"Client failed: {str(e)}")

2025-05-11 15:01:07,185 - FraminghamClient - INFO - Using device: cpu
2025-05-11 15:01:07,198 - FraminghamClient - INFO - Loaded framingham_part2.csv with shape (1060, 16)
2025-05-11 15:01:07,205 - FraminghamClient - INFO - Class distribution: {0: 914, 1: 146}
2025-05-11 15:01:07,214 - FraminghamClient - INFO - Created dataloader with 1060 samples and 15 features


2025-05-11 15:01:07,219 - FraminghamClient - INFO - Model initialized with input size: 15
2025-05-11 15:01:07,223 - FraminghamClient - INFO - Initialized client with device: cpu
2025-05-11 15:01:07,226 - FraminghamClient - INFO - Starting client 1 and connecting to localhost:8080
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
2025-05-11 15:01:07,231 - flwr - WARNING - DEPRECATED FEATURE: flwr.client.start_client() is deprecated.
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is 

Running in Jupyter/IPython environment

===== Framingham Heart Study FL Client 1 (FedProx) =====
Server:        localhost:8080
Data file:     framingham_part2.csv
Local epochs:  3
Batch size:    32
Proximal mu:   0.01
Device:        cpu

Connecting to server...



INFO :      
2025-05-11 15:01:10,560 - flwr - INFO - 
INFO :      Received: train message 2abc3f7d-e072-422d-ad8b-d6402656c1b0
2025-05-11 15:01:10,566 - flwr - INFO - Received: train message 2abc3f7d-e072-422d-ad8b-d6402656c1b0
2025-05-11 15:01:10,566 - flwr - WARNING - Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
2025-05-11 15:01:10,574 - FraminghamClient - INFO - Parameters updated from server
2025-05-11 15:01:10,590 - FraminghamClient - INFO - Epoch 1/3 - Batch 0/34 - Loss: 0.7168
2025-05-11 15:01:10,660 - FraminghamClient - INFO - Epoch 1/3 - Batch 5/34 - Loss: 0.6781
2025-05-11 15:01:10,732 - FraminghamClient - INFO - Epoch 1/3 - Batch 10/34 - Loss: 0.6662
2025-05-11 15:01:10,796 - FraminghamClient - INFO - Epoch 1/3 - Batch 15/34 - Loss: 0.6410
2025-05-11 15:01:10,857 - FraminghamClient - INFO - Epoch 1/3 - Batch 20/34 - Loss:

In [30]:
# improved_client.py
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import os
import sys
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("ImprovedFraminghamClient")

# Client configuration
CLIENT_CONFIG = {
    "local_epochs": 3,
    "batch_size": 32,
    "learning_rate": 0.001,
    "weight_decay": 1e-5,
    "dropout_rate": 0.3,
    "server_address": "localhost:8080",
    "proximal_mu": 0.01,        # FedProx hyperparameter
    "pos_weight": 5.0,          # Base positive class weight (will be adjusted dynamically)
    "focal_loss_gamma": 2.0,    # Focal loss focusing parameter
    "focal_loss_alpha": 0.25,   # Focal loss alpha parameter
    "use_focal_loss": True,     # Use focal loss instead of weighted BCE
    "use_smote": True,          # Use SMOTE for balanced sampling
    "evaluation_threshold": 0.30  # Lower threshold for evaluation
}

# Model for Framingham Heart Study data
class HeartDiseaseModel(nn.Module):
    def __init__(self, input_size):
        super(HeartDiseaseModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(CLIENT_CONFIG["dropout_rate"]),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(CLIENT_CONFIG["dropout_rate"]),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

# Weighted BCE Loss for class imbalance
class WeightedBCELoss(nn.Module):
    def __init__(self, pos_weight=5.0):
        super(WeightedBCELoss, self).__init__()
        self.pos_weight = pos_weight
        
    def forward(self, y_pred, y_true):
        # Create weight tensor based on true labels
        weights = torch.where(y_true == 1.0, 
                             self.pos_weight * torch.ones_like(y_true),
                             torch.ones_like(y_true))
        
        # Binary cross entropy loss
        bce = -(y_true * torch.log(y_pred + 1e-7) + (1 - y_true) * torch.log(1 - y_pred + 1e-7))
        
        # Apply weights and mean reduction
        return (weights * bce).mean()

# Focal Loss for harder focus on misclassified examples
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha  # Class balancing parameter
        self.gamma = gamma  # Focusing parameter
        
    def forward(self, y_pred, y_true):
        # BCE loss
        bce = -(y_true * torch.log(y_pred + 1e-7) + (1 - y_true) * torch.log(1 - y_pred + 1e-7))
        
        # Probability of the prediction being correct
        pt = torch.where(y_true == 1, y_pred, 1 - y_pred)
        
        # Apply focusing parameter to down-weight easy examples
        focal_weight = (1 - pt) ** self.gamma
        
        # Apply alpha for class balancing
        alpha_weight = torch.where(y_true == 1, 
                                  self.alpha * torch.ones_like(y_true),
                                  (1 - self.alpha) * torch.ones_like(y_true))
        
        # Combine all weights
        loss = alpha_weight * focal_weight * bce
        return loss.mean()

# FedProx Loss Function
class FedProxLoss(nn.Module):
    def __init__(self, base_criterion, mu=0.01):
        super(FedProxLoss, self).__init__()
        self.base_criterion = base_criterion
        self.mu = mu  # Proximal term coefficient
        
    def forward(self, y_pred, y_true, model_params, global_params):
        # Calculate the base loss (e.g., Focal loss or weighted BCE)
        base_loss = self.base_criterion(y_pred, y_true)
        
        # Calculate the proximal term if global parameters are provided
        proximal_term = 0.0
        if global_params is not None:
            # Sum up the squared L2 norm of the difference between local and global model parameters
            for local_param, global_param in zip(model_params, global_params):
                proximal_term += torch.sum((local_param - global_param) ** 2)
                
            # Add the weighted proximal term to the base loss
            loss = base_loss + (self.mu / 2) * proximal_term
            return loss
        
        # If no global parameters are provided, just return the base loss
        return base_loss

# Load and preprocess Framingham data with optional SMOTE
def load_data(data_path):
    """Load and preprocess Framingham Heart Study data with optional SMOTE balancing"""
    try:
        # Read CSV data
        df = pd.read_csv(data_path)
        logger.info(f"Loaded {data_path} with shape {df.shape}")
        
        # Check for missing values
        missing_values = df.isnull().sum().sum()
        if missing_values > 0:
            logger.info(f"Found {missing_values} missing values, dropping rows with missing values")
            df.dropna(inplace=True)
            logger.info(f"Shape after dropping missing values: {df.shape}")
        
        # Ensure the target column exists
        if "TenYearCHD" not in df.columns:
            raise ValueError("Target column 'TenYearCHD' not found in dataset!")
            
        # Split features and target
        X = df.drop(columns=["TenYearCHD"])
        y = df["TenYearCHD"]
        
        # Calculate class weights based on distribution
        class_counts = y.value_counts()
        logger.info(f"Original class distribution: {class_counts.to_dict()}")
        
        # Calculate weight for positive class (inverse of frequency)
        if 1 in class_counts and 0 in class_counts:
            neg_count = class_counts[0]
            pos_count = class_counts[1]
            # Use aggressive weighting - 2x the standard ratio
            pos_weight = (neg_count / pos_count) * 2.0
            CLIENT_CONFIG["pos_weight"] = pos_weight
            logger.info(f"Set positive class weight to: {pos_weight:.4f}")
            
            # Also adjust focal loss alpha based on class distribution
            # Alpha should be higher for more imbalanced datasets
            imbalance_ratio = pos_count / (pos_count + neg_count)
            CLIENT_CONFIG["focal_loss_alpha"] = max(0.25, 1 - imbalance_ratio)
            logger.info(f"Set focal loss alpha to: {CLIENT_CONFIG['focal_loss_alpha']:.4f}")
        
        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Apply SMOTE for balancing if enabled
        if CLIENT_CONFIG["use_smote"]:
            try:
                logger.info("Applying SMOTE for class balancing...")
                smote = SMOTE(random_state=42)
                X_resampled, y_resampled = smote.fit_resample(X_scaled, y)
                
                # Log the new distribution
                unique, counts = np.unique(y_resampled, return_counts=True)
                logger.info(f"After SMOTE class distribution: {dict(zip(unique, counts))}")
                
                X_scaled = X_resampled
                y = y_resampled
            except Exception as e:
                logger.error(f"SMOTE failed: {str(e)}. Using original imbalanced data.")
        
        # Convert to tensors
        X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)
        
        # Create dataset and dataloader
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(dataset, batch_size=CLIENT_CONFIG["batch_size"], shuffle=True)
        
        logger.info(f"Created dataloader with {len(dataset)} samples and {X.shape[1]} features")
        return dataloader, X.shape[1]
    
    except Exception as e:
        logger.error(f"Error loading data: {str(e)}")
        raise

# Client class for Federated Learning with FedProx
class ImprovedFraminghamClient(fl.client.NumPyClient):
    def __init__(self, model, dataloader, device):
        self.model = model
        self.dataloader = dataloader
        self.device = device
        self.global_params = None  # Store global model parameters for FedProx
        logger.info(f"Initialized client with device: {device}")
        
    def get_parameters(self, config):
        """Get model parameters as a list of NumPy arrays"""
        # Using detach() to prevent gradient error
        return [val.detach().cpu().numpy() for val in self.model.parameters()]
    
    def set_parameters(self, parameters):
        """Set model parameters from a list of NumPy arrays"""
        # Convert to torch tensors
        self.global_params = [torch.tensor(p, device=self.device) for p in parameters]
        
        # Update model
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(v, device=self.device) for k, v in params_dict}
        self.model.load_state_dict(state_dict, strict=True)
        logger.info("Parameters updated from server")
        
    def fit(self, parameters, config):
        """Train the model on local data with FedProx"""
        # Update model with server parameters
        self.set_parameters(parameters)
        
        # Train the model
        self.model.train()
        
        # Select loss function
        if CLIENT_CONFIG["use_focal_loss"]:
            criterion = FocalLoss(
                alpha=CLIENT_CONFIG["focal_loss_alpha"], 
                gamma=CLIENT_CONFIG["focal_loss_gamma"]
            )
            logger.info(f"Using Focal Loss with alpha={CLIENT_CONFIG['focal_loss_alpha']}, gamma={CLIENT_CONFIG['focal_loss_gamma']}")
        else:
            criterion = WeightedBCELoss(pos_weight=CLIENT_CONFIG["pos_weight"])
            logger.info(f"Using Weighted BCE Loss with pos_weight={CLIENT_CONFIG['pos_weight']}")
        
        # FedProx loss function
        proximal_criterion = FedProxLoss(criterion, mu=CLIENT_CONFIG["proximal_mu"])
        
        optimizer = torch.optim.Adam(
            self.model.parameters(), 
            lr=CLIENT_CONFIG["learning_rate"],
            weight_decay=CLIENT_CONFIG["weight_decay"]
        )
        
        # Metrics for tracking
        total_loss = 0.0
        total_samples = 0
        correct = 0
        true_positives = 0
        true_negatives = 0
        predicted_positives = 0
        actual_positives = 0
        
        # Train for multiple epochs
        for epoch in range(CLIENT_CONFIG["local_epochs"]):
            epoch_loss = 0.0
            epoch_samples = 0
            
            for batch_idx, (X, y) in enumerate(self.dataloader):
                # Move tensors to device
                X, y = X.to(self.device), y.to(self.device)
                
                # Forward pass
                y_pred = self.model(X)
                
                # Calculate loss with proximal term
                loss = proximal_criterion(
                    y_pred, 
                    y, 
                    self.model.parameters(),  # Current model parameters
                    self.global_params        # Global model parameters
                )
                
                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                # Update metrics
                batch_loss = loss.item() * X.size(0)
                total_loss += batch_loss
                epoch_loss += batch_loss
                total_samples += X.size(0)
                epoch_samples += X.size(0)
                
                # Use lower threshold for positive class prediction
                threshold = CLIENT_CONFIG["evaluation_threshold"]
                predicted = (y_pred > threshold).float()
                correct += (predicted == y).sum().item()
                
                # Calculate class-specific metrics
                true_positives += ((predicted == 1) & (y == 1)).sum().item()
                true_negatives += ((predicted == 0) & (y == 0)).sum().item()
                predicted_positives += (predicted == 1).sum().item()
                actual_positives += (y == 1).sum().item()
                
                # Log progress occasionally
                if batch_idx % 5 == 0:
                    logger.info(
                        f"Epoch {epoch+1}/{CLIENT_CONFIG['local_epochs']} - "
                        f"Batch {batch_idx}/{len(self.dataloader)} - "
                        f"Loss: {loss.item():.4f}"
                    )
            
            # Log epoch metrics
            logger.info(
                f"Epoch {epoch+1}/{CLIENT_CONFIG['local_epochs']} completed - "
                f"Loss: {epoch_loss/epoch_samples:.4f}"
            )
        
        # Calculate final metrics
        avg_loss = total_loss / total_samples if total_samples > 0 else 0
        accuracy = correct / total_samples if total_samples > 0 else 0
        
        # Calculate recall (sensitivity) for positive class
        recall = true_positives / actual_positives if actual_positives > 0 else 0
        
        # Calculate precision for positive class
        precision = true_positives / predicted_positives if predicted_positives > 0 else 0
        
        # Calculate F1 score
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        logger.info(f"Training completed - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
        logger.info(f"Minority class metrics - Recall: {recall:.4f}, Precision: {precision:.4f}, F1: {f1:.4f}")
        
        # Return updated model parameters and metrics
        return self.get_parameters({}), total_samples, {
            "loss": float(avg_loss), 
            "accuracy": float(accuracy),
            "recall": float(recall),
            "precision": float(precision),
            "f1": float(f1)
        }
    
    def evaluate(self, parameters, config):
        """Evaluate the model on local data"""
        # Update model with server parameters
        self.set_parameters(parameters)
        
        # Evaluate the model
        self.model.eval()
        if CLIENT_CONFIG["use_focal_loss"]:
            criterion = FocalLoss(
                alpha=CLIENT_CONFIG["focal_loss_alpha"], 
                gamma=CLIENT_CONFIG["focal_loss_gamma"]
            )
        else:
            criterion = WeightedBCELoss(pos_weight=CLIENT_CONFIG["pos_weight"])
        
        loss = 0.0
        total = 0
        correct = 0
        true_positives = 0
        true_negatives = 0
        predicted_positives = 0
        actual_positives = 0
        
        with torch.no_grad():
            for X, y in self.dataloader:
                # Move tensors to device
                X, y = X.to(self.device), y.to(self.device)
                
                # Forward pass
                y_pred = self.model(X)
                batch_loss = criterion(y_pred, y).item()
                
                # Update metrics
                loss += batch_loss * X.size(0)
                total += X.size(0)
                
                # Use lower threshold for evaluation
                threshold = CLIENT_CONFIG["evaluation_threshold"]
                predicted = (y_pred > threshold).float()
                correct += (predicted == y).sum().item()
                
                # Calculate class-specific metrics
                actual_positives += (y == 1).sum().item()
                predicted_positives += (predicted == 1).sum().item()
                true_positives += ((predicted == 1) & (y == 1)).sum().item()
                true_negatives += ((predicted == 0) & (y == 0)).sum().item()
        
        # Calculate final metrics
        avg_loss = loss / total if total > 0 else 0
        accuracy = correct / total if total > 0 else 0
        
        # Calculate recall (sensitivity) for positive class
        recall = true_positives / actual_positives if actual_positives > 0 else 0
        
        # Calculate precision for positive class
        precision = true_positives / predicted_positives if predicted_positives > 0 else 0
        
        # Calculate F1 score
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        logger.info(f"Evaluation - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
        logger.info(f"Minority class metrics - Recall: {recall:.4f}, Precision: {precision:.4f}, F1: {f1:.4f}")
        
        # Return metrics
        return float(avg_loss), total, {
            "accuracy": float(accuracy),
            "recall": float(recall),
            "precision": float(precision),
            "f1": float(f1)
        }

def start_client(client_id=0, server_address=None):
    """Initialize and start a client"""
    # Update server address if provided
    if server_address:
        CLIENT_CONFIG["server_address"] = server_address
    
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")
    
    # Determine which data file to use based on client ID
    data_path = f"framingham_part{client_id+1}.csv"
    
    # Try alternative naming if file doesn't exist
    if not os.path.exists(data_path):
        alternative_path = f"framingham_part{client_id+1}.csv"
        if os.path.exists(alternative_path):
            data_path = alternative_path
        else:
            logger.error(f"Data file {data_path} not found")
            return
    
    # Load data
    dataloader, input_size = load_data(data_path)
    
    # Initialize model
    model = HeartDiseaseModel(input_size=input_size).to(device)
    logger.info(f"Model initialized with input size: {input_size}")
    
    # Create client
    client = ImprovedFraminghamClient(model, dataloader, device)
    
    # Start client
    logger.info(f"Starting client {client_id} and connecting to {CLIENT_CONFIG['server_address']}")
    
    print(f"\n===== Improved Framingham Heart Study FL Client {client_id} (FedProx) =====")
    print(f"Server:           {CLIENT_CONFIG['server_address']}")
    print(f"Data file:        {data_path}")
    print(f"Local epochs:     {CLIENT_CONFIG['local_epochs']}")
    print(f"Batch size:       {CLIENT_CONFIG['batch_size']}")
    print(f"Proximal mu:      {CLIENT_CONFIG['proximal_mu']}")
    print(f"Using SMOTE:      {CLIENT_CONFIG['use_smote']}")
    if CLIENT_CONFIG["use_focal_loss"]:
        print(f"Loss function:    Focal Loss (alpha={CLIENT_CONFIG['focal_loss_alpha']}, gamma={CLIENT_CONFIG['focal_loss_gamma']})")
    else:
        print(f"Loss function:    Weighted BCE (pos_weight={CLIENT_CONFIG['pos_weight']})")
    print(f"Eval threshold:   {CLIENT_CONFIG['evaluation_threshold']}")
    print(f"Device:           {device}")
    print("=================================================")
    print(f"\nConnecting to server...\n")
    
    fl.client.start_client(server_address=CLIENT_CONFIG["server_address"], client=client)

# For Jupyter usage
if __name__ == "__main__":
    # Check if running in Jupyter
    if 'ipykernel' in sys.modules:
        print("Running in Jupyter/IPython environment")
        # Default to client ID 0, can be changed by user
        start_client(client_id=1)
    else:
        # For command line use
        import argparse
        parser = argparse.ArgumentParser(description="Improved Framingham Heart Study FL Client")
        parser.add_argument("--id", type=int, default=0, help="Client ID (0, 1, or 2)")
        parser.add_argument("--server", type=str, default="localhost:8080", help="Server address")
        parser.add_argument("--mu", type=float, default=0.01, help="FedProx proximal term strength")
        parser.add_argument("--pos_weight", type=float, default=None, help="Weight for positive class")
        parser.add_argument("--focal", action="store_true", help="Use focal loss instead of weighted BCE")
        parser.add_argument("--smote", action="store_true", help="Use SMOTE for class balancing")
        parser.add_argument("--threshold", type=float, default=None, help="Evaluation threshold")
        
        args = parser.parse_args()
        
        if args.id not in [0, 1, 2]:
            logger.error("Client ID must be 0, 1, or 2")
        else:
            try:
                # Set FedProx hyperparameter
                CLIENT_CONFIG["proximal_mu"] = args.mu
                
                # Set positive class weight if provided
                if args.pos_weight is not None:
                    CLIENT_CONFIG["pos_weight"] = args.pos_weight
                
                # Set loss function
                if args.focal:
                    CLIENT_CONFIG["use_focal_loss"] = True
                
                # Set SMOTE usage
                if args.smote:
                    CLIENT_CONFIG["use_smote"] = True
                
                # Set evaluation threshold
                if args.threshold is not None:
                    CLIENT_CONFIG["evaluation_threshold"] = args.threshold
                    
                start_client(args.id, args.server)
            except Exception as e:
                logger.error(f"Client failed: {str(e)}")

2025-05-11 15:54:27,104 - ImprovedFraminghamClient - INFO - Using device: cpu
2025-05-11 15:54:27,129 - ImprovedFraminghamClient - INFO - Loaded framingham_part2.csv with shape (1060, 16)
2025-05-11 15:54:27,129 - ImprovedFraminghamClient - INFO - Original class distribution: {0: 914, 1: 146}
2025-05-11 15:54:27,137 - ImprovedFraminghamClient - INFO - Set positive class weight to: 12.5205
2025-05-11 15:54:27,139 - ImprovedFraminghamClient - INFO - Set focal loss alpha to: 0.8623
2025-05-11 15:54:27,153 - ImprovedFraminghamClient - INFO - Applying SMOTE for class balancing...
2025-05-11 15:54:27,176 - ImprovedFraminghamClient - INFO - After SMOTE class distribution: {0: 914, 1: 914}
2025-05-11 15:54:27,184 - ImprovedFraminghamClient - INFO - Created dataloader with 1828 samples and 15 features
2025-05-11 15:54:27,184 - ImprovedFraminghamClient - INFO - Model initialized with input size: 15
2025-05-11 15:54:27,192 - ImprovedFraminghamClient - INFO - Initialized client with device: cpu
20

Running in Jupyter/IPython environment

===== Improved Framingham Heart Study FL Client 1 (FedProx) =====
Server:           localhost:8080
Data file:        framingham_part2.csv
Local epochs:     3
Batch size:       32
Proximal mu:      0.01
Using SMOTE:      True
Loss function:    Focal Loss (alpha=0.8622641509433963, gamma=2.0)
Eval threshold:   0.3
Device:           cpu

Connecting to server...



INFO :      
2025-05-11 15:55:21,590 - flwr - INFO - 
INFO :      Received: train message 1a02a800-81df-4c38-8ee9-173b7712c988
2025-05-11 15:55:21,598 - flwr - INFO - Received: train message 1a02a800-81df-4c38-8ee9-173b7712c988
2025-05-11 15:55:21,606 - flwr - WARNING - Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
2025-05-11 15:55:21,614 - ImprovedFraminghamClient - INFO - Parameters updated from server
2025-05-11 15:55:21,614 - ImprovedFraminghamClient - INFO - Using Focal Loss with alpha=0.8622641509433963, gamma=2.0
2025-05-11 15:55:21,637 - ImprovedFraminghamClient - INFO - Epoch 1/3 - Batch 0/58 - Loss: 0.0834
2025-05-11 15:55:21,701 - ImprovedFraminghamClient - INFO - Epoch 1/3 - Batch 5/58 - Loss: 0.0994
2025-05-11 15:55:21,753 - ImprovedFraminghamClient - INFO - Epoch 1/3 - Batch 10/58 - Loss: 0.0799
2025-05-11 15:55:21,805 

In [ ]:
# fl_client.py

import torch
import torch.nn as nn
import pandas as pd
import pickle
import socket
import time
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

# --- 1) Model + loss must match server exactly ---
class HeartDiseaseModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 32)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.dropout1(self.relu1(self.fc1(x)))
        x = self.dropout2(self.relu2(self.fc2(x)))
        return self.sigmoid(self.fc3(x))

class WeightedBCELoss(nn.Module):
    def __init__(self, pos_weight=5.0):
        super().__init__()
        self.pos_weight = pos_weight
    def forward(self, pred, true):
        w = torch.where(true==1, self.pos_weight*torch.ones_like(true), torch.ones_like(true))
        bce = -( true*torch.log(pred+1e-7) + (1-true)*torch.log(1-pred+1e-7) )
        return (w*bce).mean()

# --- 2) Load & preprocess ---
def load_data(path):
    df = pd.read_csv(path)
    # fill nulls
    for c in df.columns:
        if df[c].isnull().any():
            df[c].fillna(df[c].median(), inplace=True)
    X = df.drop("TenYearCHD", axis=1)
    y = df["TenYearCHD"].astype(float)
    # engineered
    X["age_sq"] = X["age"]**2
    X["bp_prod"]= X["sysBP"]*X["diaBP"]
    X["smk_int"]= X["cigsPerDay"]*X["currentSmoker"]
    X["risk_sc"]= (X["age"]/10 + X["sysBP"]/40
                   + X["currentSmoker"]*2 + X["diabetes"]*3
                   + X["male"]*1.5)
    scaler=StandardScaler()
    Xs = scaler.fit_transform(X)
    xt = torch.tensor(Xs, dtype=torch.float32)
    yt = torch.tensor(y.values, dtype=torch.float32).view(-1,1)
    ds = TensorDataset(xt,yt)
    dl = DataLoader(ds,batch_size=32,shuffle=True)
    posw = y.value_counts()[0]/y.value_counts()[1]
    return dl, xt.shape[1], posw, len(ds)

# --- 3) Train step ---
def train_model(model, dl, posw, epochs=3):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device).train()
    opt = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    crit=WeightedBCELoss(posw)
    for e in range(epochs):
        tot, corr = 0.0,0
        for X,y in dl:
            X,y = X.to(device), y.to(device)
            opt.zero_grad()
            p = model(X)
            loss = crit(p,y)
            loss.backward()
            opt.step()
            tot+=loss.item()*X.size(0)
            corr+=( (p>0.3).float()==y ).sum().item()
        print(f"[Client] Epoch {e+1}/{epochs}: loss={tot/len(dl.dataset):.4f} acc={corr/len(dl.dataset):.4f}")

# --- 4) FL Client ---
class FLClient:
    def __init__(self, cid, host, port, data_path):
        self.cid, self.addr = cid, (host,port)
        self.dl, self.inp_sz, self.posw, self.nsamples = load_data(data_path)
        self.model = HeartDiseaseModel(self.inp_sz)
        self.sock = None

    def _send(self,m): self.sock.sendall(f"{m}\n".encode())
    def _recv(self):
        b=b"" 
        while True:
            c=self.sock.recv(1024)
            if not c: raise IOError("server gone")
            b+=c
            if b"\n" in b:
                line, b=b.split(b"\n",1)
                return line.decode()

    def connect(self):
        self.sock=socket.socket()
        self.sock.connect(self.addr)
        self._send(self.cid)
        time.sleep(0.1)
        self._send("PING")
        return self._recv()=="PONG"

    def get_round(self):
        self._send("GET_ROUND")
        return int(self._recv())

    def get_model(self):
        self._send("GET_MODEL")
        L=int(self._recv())
        data=b""
        while len(data)<L:
            data+=self.sock.recv(L-len(data))
        st=pickle.loads(data)
        self.model.load_state_dict(st)

    def send_model(self):
        pkg={"model":self.model.state_dict(),"num_samples":self.nsamples}
        data=pickle.dumps(pkg)
        self._send(f"SUBMIT_MODEL:{len(data)}")
        if self._recv()=="READY":
            self.sock.sendall(data)
            return self._recv()=="SUCCESS"
        return False

    def run(self, rounds=10):
        if not self.connect():
            print("[Client] cannot connect."); return
        last=-1
        while True:
            r=self.get_round()
            if r>rounds: break
            if r==last:
                time.sleep(2); continue
            last=r
            print(f"[Client] Round {r}/{rounds}")
            self.get_model()
            train_model(self.model, self.dl, self.posw, epochs=3)
            if not self.send_model():
                print("[Client] submit failed")
            time.sleep(1)
        print("[Client] Done.")

if __name__=="__main__":
    # change CLIENT_ID and DATA_PATH per instance
    client = FLClient("client_1","localhost",8765,"framingham_part2.csv")
    client.run(rounds=10)


[Client] Round 0/10
[Client] Epoch 1/3: loss=1.1883 acc=0.1377
[Client] Epoch 2/3: loss=1.1551 acc=0.1377
[Client] Epoch 3/3: loss=1.1254 acc=0.1377
